# MNIST Classifier with PyTorch framework

* Based on http://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html
* Anton Karazeev, you can text me: [```anton.karazeev@gmail.com```](mailto:anton.karazeev@phystech.edu) or [t.me/akarazeev](https://t.me/akarazeev)

In [ ]:
%matplotlib inline

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
# Download MNIST dataset
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                      download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                     download=True, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Function to show an image
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    
# Get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# Show images
imshow(torchvision.utils.make_grid(images))
# Print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

# Training

In [ ]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

## Network definition

In [ ]:
# Define our net's class
class Net(nn.Module):
    def __init__(self):
        # Hint: things like `print(self.conv1(x).size())` in `forward()`
        # can help you to find correct layers' parameters
        super(Net, self).__init__()
        
        # set <in_channels>, <out_channels>, <kernel_size>
        self.conv1 = nn.Conv2d(<in_channels>, <out_channels>, <kernel_size>)
        
        # set <kernel_size> and <stride> here
        self.pool = nn.MaxPool2d(<kernel_size>, <stride>)
        
        # set correct <input_size> here
        # it equals to (Height * Width * 'Number of channels')
        self.fc1 = nn.Linear(<input_size>, 80)
        self.fc2 = nn.Linear(80, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()

## Choose loss function

Use [Cross Entropy](https://en.wikipedia.org/wiki/Cross_entropy) as a loss function: $H(p,q)=-\sum_i p_i \cdot \log(q_i)$, where $p_i$ - true label, $q_i$ - prediction.

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
# set <learning_rate> and <momentum> here
optimizer = optim.SGD(net.parameters(), <learning_rate>, <momentum>)

## Let's train our net

In [ ]:
num_epochs = 2

for epoch in range(num_epochs):  # Loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs
        inputs, labels = data

        # Wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward -> backward -> optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:  # Print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

# Testing the network

## Ground Truth

In [ ]:
# Get some random testing images
dataiter = iter(testloader)
images, labels = dataiter.next()

# Print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

## Prediction

Higher the energy for a class - the more net thinks that the image is of a particular class.

In [ ]:
outputs = net(Variable(images))
outputs  # Energies for every class

So we should get the index of maximum energy for every image.

In [ ]:
_, predicted = torch.max(outputs.data, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

## Measure the Accuracy

In [ ]:
correct = 0
total = 0

for data in testloader:
    images, labels = data
    outputs = net(Variable(images))            # Prediction
    _, predicted = torch.max(outputs.data, 1)  # Indices of max energies
    total += labels.size(0)                    # Number of samples in `data`
    correct += (predicted == labels).sum()     # Number of right predictions

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

## Accuracy for every class

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

for data in testloader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels).squeeze()
    for i in range(4):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1

for i in range(10):
    print('Accuracy of %2s is %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))